In [1]:
from google.colab import drive
drive.mount('/drive')
%cd /drive

Mounted at /drive
/drive


In [2]:
%cd MyDrive

/drive/MyDrive


In [3]:
!pip install number_parser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.2 MB/s 


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.7 MB/s 
     |████████████████████████████████| 6.6 MB 52.0 MB/s 
     |████████████████████████████████| 101 kB 15.5 MB/s 
     |████████████████████████████████| 596 kB 55.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
from main_bot import *

In [6]:
dataset_preproc = DatasetPreprocessor()

In [7]:
import tensorflow as tf
from transformers import AutoTokenizer

model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [8]:
'''
text = dataset_preproc.read_file('dataset/dataset/snips/train/seq.in')
val_text = dataset_preproc.read_file('dataset/dataset/snips/dev/seq.in')
text.extend(val_text)
encoded_texts = dataset_preproc.encode_texts(tokenizer, text)
'''
intent_labels = dataset_preproc.read_intent_labels('dataset/dataset/snips/intent_label.txt')
slot_label = dataset_preproc.preprocess_slot_labels('dataset/dataset/snips/slot_label.txt')
intent_map = dataset_preproc.get_intent_map(intent_labels)
slot_map = dataset_preproc.get_slot_map(slot_label)

'''
intents = dataset_preproc.read_file('dataset/dataset/snips/train/label')
val_intents = dataset_preproc.read_file('dataset/dataset/snips/dev/label')
intents.extend(val_intents)

encoded_intents = dataset_preproc.encode_intents(intents, intent_map)

slots = dataset_preproc.read_file('dataset/dataset/snips/train/seq.out')
val_slots = dataset_preproc.read_file('dataset/dataset/snips/dev/seq.out')
slots.extend(val_slots)

slots_dict = dataset_preproc.get_slot_indexes(text, slots)
val_slots_dict = dataset_preproc.get_slot_indexes(text, val_slots)

max_len = len(encoded_texts["input_ids"][0])
encoded_slots = dataset_preproc.encode_slots(slots_dict, text, tokenizer, slot_map, max_len)
'''

'\nintents = dataset_preproc.read_file(\'dataset/dataset/snips/train/label\')\nval_intents = dataset_preproc.read_file(\'dataset/dataset/snips/dev/label\')\nintents.extend(val_intents)\n\nencoded_intents = dataset_preproc.encode_intents(intents, intent_map)\n\nslots = dataset_preproc.read_file(\'dataset/dataset/snips/train/seq.out\')\nval_slots = dataset_preproc.read_file(\'dataset/dataset/snips/dev/seq.out\')\nslots.extend(val_slots)\n\nslots_dict = dataset_preproc.get_slot_indexes(text, slots)\nval_slots_dict = dataset_preproc.get_slot_indexes(text, val_slots)\n\nmax_len = len(encoded_texts["input_ids"][0])\nencoded_slots = dataset_preproc.encode_slots(slots_dict, text, tokenizer, slot_map, max_len)\n'

In [9]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint


joint_model = JointIntentAndSlotFillingModel(model_name=model_name, intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))

opt = Adam(learning_rate=3e-5, epsilon=1e-08)

# two outputs, one for slots, another for intents
# we have to fine tune for both
losses = [SparseCategoricalCrossentropy(from_logits=True),
          SparseCategoricalCrossentropy(from_logits=True)]

metrics = [SparseCategoricalAccuracy("accuracy")]
# compile model
joint_model.compile(optimizer=opt, loss=losses, metrics=metrics)

#joint_model = JointIntentAndSlotFillingModel(intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))

joint_model(tf.constant(tokenizer.encode("what is the weather in torre del greco"))[None, :])
joint_model.load_weights('joint_model.h5')

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [10]:
dialog = Dialog("user", 0, joint_model, intent_labels, slot_label, tokenizer)

In [11]:
from GUI import *
view = GUI(dialog)
view.initGUI()

Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/162 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


IntSlider(value=5, continuous_update=False, description='Duration (s):', max=20, min=1)

TwoByTwoLayout(children=(Button(description='Record', icon='microphone-lines', layout=Layout(grid_area='top-le…

TwoByTwoLayout(children=(Button(description='Export Graph', icon='check', layout=Layout(grid_area='top-left', …

# Grafo in formato graphml

In [12]:
dialog.save_graph()

# Test e metriche

In [13]:
# Prova accuracy keras
text = dataset_preproc.read_file('dataset/dataset/snips/test/seq.in')
encoded_texts = dataset_preproc.encode_texts(tokenizer, text)
intents = dataset_preproc.read_file('dataset/dataset/snips/test/label')
encoded_intents = dataset_preproc.encode_intents(intents, intent_map)
slots = dataset_preproc.read_file('dataset/dataset/snips/test/seq.out')
slots_dict = dataset_preproc.get_slot_indexes(text, slots)
max_len = len(encoded_texts["input_ids"][0])
encoded_slots = dataset_preproc.encode_slots(slots_dict, text, tokenizer, slot_map, max_len)

'''x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}
m = SparseCategoricalAccuracy("accuracy")
m.reset_state()
m.update_state(encoded_intents, joint_model(x)[1])
print("Acc intents: " + str(m.result().numpy()))
m.reset_state()
m.update_state(encoded_slots, joint_model(x)[0])
print("Acc slots: " + str(m.result().numpy()))'''

'x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}\nm = SparseCategoricalAccuracy("accuracy")\nm.reset_state()\nm.update_state(encoded_intents, joint_model(x)[1])\nprint("Acc intents: " + str(m.result().numpy()))\nm.reset_state()\nm.update_state(encoded_slots, joint_model(x)[0])\nprint("Acc slots: " + str(m.result().numpy()))'

In [14]:
def getScores():
  x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}
  MetricsCalculator.accuracy_metrics(encoded_intents, encoded_slots, x, joint_model)
  MetricsCalculator.precision_recall_fscore_metrics(encoded_intents, encoded_slots, x, joint_model)

Acc intents: 0.9771429

Acc slots: 0.98433036

Precision Recall F1 intents: (0.9798021530628027, 0.9769695909393802, 0.977816348507086)

Precision Recall F1 slots: (0.9420341538384924, 0.9577461780052603, 0.9477674553550474)